In [1]:
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Quadro RTX 6000, 22698 MiB, 12137 MiB
Quadro RTX 6000, 22698 MiB, 12820 MiB
Quadro RTX 6000, 22698 MiB, 19222 MiB
Quadro RTX 6000, 22698 MiB, 19238 MiB
Quadro RTX 6000, 22698 MiB, 19244 MiB
Quadro RTX 6000, 22698 MiB, 8250 MiB
Quadro RTX 6000, 22698 MiB, 21860 MiB
Quadro RTX 6000, 22698 MiB, 22128 MiB


In [2]:
%pip install jupyter_compare_view

     |████████████████████████████████| 133 kB 29.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [18]:
# Fill in these environment variables
%env MODEL_NAME=stabilityai/stable-diffusion-2-depth
%env INSTANCE_DIR=/workspace/Lab/DCT-Net/DCTNet/datasets/limjukyung_aligned_512
%env CLASS_DIR=/workspace/Lab/DCT-Net/DCTNet/datasets/ffhq_512_100
%env OUTPUT_DIR=/workspace/Lab/dreambooth_depth2img/output
%load_ext autoreload

env: MODEL_NAME=stabilityai/stable-diffusion-2-depth
env: INSTANCE_DIR=/workspace/Lab/DCT-Net/DCTNet/datasets/limjukyung_aligned_512
env: CLASS_DIR=/workspace/Lab/DCT-Net/DCTNet/datasets/ffhq_512_100
env: OUTPUT_DIR=/workspace/Lab/dreambooth_depth2img/output
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
%autoreload 2

# !python -m debugpy --listen 0.0.0.0:5678 --wait-for-client \
!CUDA_VISIBLE_DEVICES=6,7 accelerate launch \
  train_dreambooth.py \
  --mixed_precision="fp16" \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --pretrained_txt2img_model_name_or_path="stabilityai/stable-diffusion-2-1-base" \
  --train_text_encoder \
  --instance_data_dir=$INSTANCE_DIR \
  --class_data_dir=$CLASS_DIR \
  --output_dir=$OUTPUT_DIR \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --instance_prompt="a photo of limjukyung person face" \
  --class_prompt="a photo of person face" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --learning_rate=1e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=200 \
  --max_train_steps=300 \
  --use_8bit_adam

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
/opt/conda/envs/ldm/lib/python3.8/site-packages/accelerate/accelerator.py:231: FutureWarning: `logging_dir` is deprecated and will be removed in version 0.18.0 of 🤗 Accelerate. Use `project_dir` instead.
  warnings.warn(
/opt/conda/envs/ldm/lib/python3.8/site-packages/accelerate/accelerator.py:336: UserWarning: `log_with=tensorboard` was passed but no supported trackers are currently installed.
  warnings.warn(f"`log_with={log_with}` was passed but no supported trackers are currently installed.")
You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all config

# Test the model you just made

In [2]:
import PIL
import torch
from torchvision import transforms
import diffusers
import transformers
from diffusers import StableDiffusionDepth2ImgPipeline
import os
from jupyter_compare_view import compare

In [3]:
print(f'Getting model from {os.environ.get("OUTPUT_DIR")}')
pipeline = StableDiffusionDepth2ImgPipeline.from_pretrained(os.environ.get('OUTPUT_DIR'))
pipeline = pipeline.to("cuda")

In [5]:
# Use an image as an input depth map
image_path = "/workspace/content/data/samples/village.jpg" # replace with whatever you want
image = PIL.Image.open(image_path)

image_transform = transforms.Compose(
    [
        transforms.Resize((384, 384)),
        transforms.ToTensor()
    ]
)
image = image_transform(image)
image = image[None,:,:,:]
image = image.to("cuda")
depth_map = pipeline.depth_estimator(image).predicted_depth
image = transforms.ToPILImage()(image[0])
depth_min = torch.amin(depth_map, dim=[0, 1, 2], keepdim=True)
depth_max = torch.amax(depth_map, dim=[0, 1, 2], keepdim=True)
depth_map = 2.0 * (depth_map - depth_min) / (depth_max - depth_min) - 1.0
depth_map = depth_map[0,:,:]
depth_map = transforms.ToPILImage()(depth_map)
compare(depth_map, image, cmap="gray", start_mode="horizontal", start_slider_pos=0.73)

In [ ]:
result = pipeline("a photo of <your instance>, standing, Kodachrome, Canon 5D, f2 aperture, extremely detailed, sharp focus", image)
compare(result[0][0], image, cmap="gray", start_mode="horizontal", start_slider_pos=0.73)